In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date, lit, date_format

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Credit") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 18:00:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
credit = spark.read.csv("/data/raw/credit_card_balance.csv",
                               header=True,
                               inferSchema=True)

credit.createOrReplaceTempView("credit")

# Contagem de linhas e colunas
num_rows = credit.count()
num_columns = len(credit.columns)

print(f'Quantidade de linhas: {num_rows}')
print(f'Quantidade de variaveis (colunas): {num_columns}')

credit.show(5, truncate=False)

Quantidade de linhas: 3840312
Quantidade de variaveis (colunas): 23
+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|


## Função para renomear e padronizar palavras

In [3]:
#Substituir espaço por underline e colocar a palavra em letras maiúsculas.
import re

def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()

    return palavra_transformada

## Renomeando e padronizando palavras da coluna NAME_CONTRACT_STATUS

In [4]:
# Extraindo os valores únicos diretamente do DataFrame
status = credit.select('NAME_CONTRACT_STATUS').distinct().collect()

# Aplicando a função de tratamento de palavra a cada valor único
nova_lista = [tratamento_palavra(row['NAME_CONTRACT_STATUS']) for row in status]

# Exibindo a lista resultante
print(nova_lista)

[Stage 6:>                                                          (0 + 8) / 8]

['DEMAND', 'COMPLETED', 'ACTIVE', 'SIGNED', 'SENT_PROPOSAL', 'REFUSED', 'APPROVED']


## Criando variáveis de janela temporal

In [5]:
df_temp_01 = spark.sql('''
SELECT
    *,
    CASE WHEN MONTHS_BALANCE >= -3 THEN 1 ELSE 0 END AS U3M,
    CASE WHEN MONTHS_BALANCE >= -6 THEN 1 ELSE 0 END AS U6M,
    CASE WHEN MONTHS_BALANCE >= -12 THEN 1 ELSE 0 END AS U12M
FROM
    credit
ORDER BY
    SK_ID_PREV
''')

df_temp_01.createOrReplaceTempView('df_temp_01')
display(df_temp_01.limit(5))

25/07/21 18:01:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[SK_ID_PREV: int, SK_ID_CURR: int, MONTHS_BALANCE: int, AMT_BALANCE: double, AMT_CREDIT_LIMIT_ACTUAL: int, AMT_DRAWINGS_ATM_CURRENT: double, AMT_DRAWINGS_CURRENT: double, AMT_DRAWINGS_OTHER_CURRENT: double, AMT_DRAWINGS_POS_CURRENT: double, AMT_INST_MIN_REGULARITY: double, AMT_PAYMENT_CURRENT: double, AMT_PAYMENT_TOTAL_CURRENT: double, AMT_RECEIVABLE_PRINCIPAL: double, AMT_RECIVABLE: double, AMT_TOTAL_RECEIVABLE: double, CNT_DRAWINGS_ATM_CURRENT: double, CNT_DRAWINGS_CURRENT: int, CNT_DRAWINGS_OTHER_CURRENT: double, CNT_DRAWINGS_POS_CURRENT: double, CNT_INSTALMENT_MATURE_CUM: double, NAME_CONTRACT_STATUS: string, SK_DPD: int, SK_DPD_DEF: int, U3M: int, U6M: int, U12M: int]

## Criando variáveis explicativas de primeira camada

In [6]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M','U12M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  # Verifica se a coluna atual não é uma coluna de flag.
  if not any(flag in coluna for flag in colunas_flags):
    for flag in colunas_flags:
      if 'DPD' in coluna:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_CRED_CARD'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_CRED_CARD'))

      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_CRED_CARD'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_CRED_CARD'))


# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
display(df_temp_02.limit(5))

Quantidade Total de Variáveis Criadas: 114
Nomes das Variáveis Criadas: ['VL_TOT_AMT_BALANCE_U3M_CRED_CARD', 'VL_MED_AMT_BALANCE_U3M_CRED_CARD', 'VL_TOT_AMT_BALANCE_U6M_CRED_CARD', 'VL_MED_AMT_BALANCE_U6M_CRED_CARD', 'VL_TOT_AMT_BALANCE_U12M_CRED_CARD', 'VL_MED_AMT_BALANCE_U12M_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_CURRENT_U3M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_CURRENT_U3M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_CURRENT_U6M_

Quantidade de linhas do DataFrame: 104307
Quantidade de colunas do DataFrame: 115




DataFrame[SK_ID_PREV: int, VL_TOT_AMT_BALANCE_U3M_CRED_CARD: double, VL_MED_AMT_BALANCE_U3M_CRED_CARD: double, VL_TOT_AMT_BALANCE_U6M_CRED_CARD: double, VL_MED_AMT_BALANCE_U6M_CRED_CARD: double, VL_TOT_AMT_BALANCE_U12M_CRED_CARD: double, VL_MED_AMT_BALANCE_U12M_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_CURRENT_U3M_CRED_CARD: double, VL_MED_AMT

## Criando variáveis com base na coluna NAME_CONTRACT_STATUS

In [7]:
# Definindo as colunas para agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for categoria in nova_lista:
  for coluna in colunas_agregacao_total:
    # Verifica se a coluna atual não é uma coluna de flag.
    if not any(flag in coluna for flag in colunas_flags):
      for flag in colunas_flags:
        if 'DPD' in coluna:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_CRED_CARD'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_{categoria}_CRED_CARD'))

        else:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_CRED_CARD'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_CRED_CARD'))
 


# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_03 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_03.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_03.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_03.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_03.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
display(df_temp_03.limit(5))

Quantidade Total de Variáveis Criadas: 798
Nomes das Variáveis Criadas: ['VL_TOT_AMT_BALANCE_U3M_DEMAND_CRED_CARD', 'VL_MED_AMT_BALANCE_U3M_DEMAND_CRED_CARD', 'VL_TOT_AMT_BALANCE_U6M_DEMAND_CRED_CARD', 'VL_MED_AMT_BALANCE_U6M_DEMAND_CRED_CARD', 'VL_TOT_AMT_BALANCE_U12M_DEMAND_CRED_CARD', 'VL_MED_AMT_BALANCE_U12M_DEMAND_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_DEMAND_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_DEMAND_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U6M_DEMAND_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U6M_DEMAND_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U12M_DEMAND_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U12M_DEMAND_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U3M_DEMAND_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U3M_DEMAND_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U6M_DEMAND_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U6M_DEMAND_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U12M_DEMAND_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U12M_DEMAND_CRED_CARD

[Stage 15:==============>                                           (2 + 6) / 8]

Quantidade de linhas do DataFrame: 104307
Quantidade de colunas do DataFrame: 799




DataFrame[SK_ID_PREV: int, VL_TOT_AMT_BALANCE_U3M_DEMAND_CRED_CARD: double, VL_MED_AMT_BALANCE_U3M_DEMAND_CRED_CARD: double, VL_TOT_AMT_BALANCE_U6M_DEMAND_CRED_CARD: double, VL_MED_AMT_BALANCE_U6M_DEMAND_CRED_CARD: double, VL_TOT_AMT_BALANCE_U12M_DEMAND_CRED_CARD: double, VL_MED_AMT_BALANCE_U12M_DEMAND_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_DEMAND_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_DEMAND_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U6M_DEMAND_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U6M_DEMAND_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U12M_DEMAND_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U12M_DEMAND_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U3M_DEMAND_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U3M_DEMAND_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U6M_DEMAND_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U6M_DEMAND_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U12M_DEMAND_CRED_CARD: double

## Realizando o join das tabelas criadas

In [8]:
# Fazendo o join das duas tabelas criadas.
df_temp_04 = df_temp_02.join(df_temp_03, 'SK_ID_PREV')



# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_04.columns
nomes_cols_novas = nomes_cols[1:-2]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_04.columns) - 3)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_04.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_04.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
display(df_temp_04.limit(5))

Quantidade Total de Variáveis Criadas: 910
Nomes das Variáveis Criadas: ['VL_TOT_AMT_BALANCE_U3M_CRED_CARD', 'VL_MED_AMT_BALANCE_U3M_CRED_CARD', 'VL_TOT_AMT_BALANCE_U6M_CRED_CARD', 'VL_MED_AMT_BALANCE_U6M_CRED_CARD', 'VL_TOT_AMT_BALANCE_U12M_CRED_CARD', 'VL_MED_AMT_BALANCE_U12M_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD', 'VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD', 'VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_CURRENT_U3M_CRED_CARD', 'VL_MED_AMT_DRAWINGS_CURRENT_U3M_CRED_CARD', 'VL_TOT_AMT_DRAWINGS_CURRENT_U6M_

[Stage 23:================>                                         (2 + 5) / 7]

Quantidade de linhas do DataFrame: 104307
Quantidade de colunas do DataFrame: 913




DataFrame[SK_ID_PREV: int, VL_TOT_AMT_BALANCE_U3M_CRED_CARD: double, VL_MED_AMT_BALANCE_U3M_CRED_CARD: double, VL_TOT_AMT_BALANCE_U6M_CRED_CARD: double, VL_MED_AMT_BALANCE_U6M_CRED_CARD: double, VL_TOT_AMT_BALANCE_U12M_CRED_CARD: double, VL_MED_AMT_BALANCE_U12M_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U3M_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U6M_CRED_CARD: double, VL_TOT_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD: bigint, VL_MED_AMT_CREDIT_LIMIT_ACTUAL_U12M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U3M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U6M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD: double, VL_MED_AMT_DRAWINGS_ATM_CURRENT_U12M_CRED_CARD: double, VL_TOT_AMT_DRAWINGS_CURRENT_U3M_CRED_CARD: double, VL_MED_AMT

In [9]:
(df_temp_04.repartition(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")
     .parquet('/data/books/credit-card'))

In [10]:
spark.stop()